This is my python file for my model

In [115]:
# Mounting my Google Drive
# comment out if using on VsCode
from google.colab import drive
import os

mountPath = '/content/drive'
drive.mount(mountPath, force_remount=True)#Forcing remount if needed

Mounted at /content/drive


# Chapter 1 - downloading and altering the first Dataset

In [116]:
#cloning full github rather than just folders, this means everything should be up to date in content
%cd /content
!rm -rf DementiaDetection
!git clone https://github.com/Unbreakable60000/DementiaDetection.git
%cd DementiaDetection

/content
Cloning into 'DementiaDetection'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 125 (delta 57), reused 47 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (125/125), 7.89 MiB | 13.15 MiB/s, done.
Resolving deltas: 100% (57/57), done.
/content/DementiaDetection


Dataset before alteration can be accessed here:
https://zenodo.org/records/7622128

In [117]:
import pandas as pd

activityDF = pd.read_csv("/content/drive/MyDrive/synoptic/oldDataset/Activity.csv")
demographicsDF = pd.read_csv("/content/drive/MyDrive/synoptic/oldDataset/Demographics.csv")
labelsDf = pd.read_csv("/content/drive/MyDrive/synoptic/oldDataset/Labels.csv")
physiologyDf = pd.read_csv("/content/drive/MyDrive/synoptic/oldDataset/Physiology.csv")
sleepDf = pd.read_csv("/content/drive/MyDrive/synoptic/oldDataset/Sleep.csv")

dfItems = {"activityDf": activityDF,"demographicsDF": demographicsDF,"labelsDf": labelsDf,"physiologyDf": physiologyDf, "sleepDf":sleepDf}

In [118]:
for name,df in dfItems.items():#itterates through list and gives column names and how many unique values are in each
    print(f"{name}:")
    print("--------")
    print(df.columns.tolist())
    for col in df.columns:
        num_unique = df[col].nunique()
        print(f"{col}: {num_unique} unique values")
    print()

activityDf:
--------
['patient_id', 'location_name', 'date']
patient_id: 56 unique values
location_name: 8 unique values
date: 916821 unique values

demographicsDF:
--------
['patient_id', 'age', 'sex']
patient_id: 56 unique values
age: 3 unique values
sex: 2 unique values

labelsDf:
--------
['patient_id', 'date', 'type']
patient_id: 49 unique values
date: 518 unique values
type: 6 unique values

physiologyDf:
--------
['patient_id', 'date', 'device_type', 'value', 'unit']
patient_id: 55 unique values
date: 9773 unique values
device_type: 8 unique values
value: 2686 unique values
unit: 5 unique values

sleepDf:
--------
['patient_id', 'date', 'state', 'heart_rate', 'respiratory_rate', 'snoring']
patient_id: 17 unique values
date: 92180 unique values
state: 4 unique values
heart_rate: 68 unique values
respiratory_rate: 24 unique values
snoring: 2 unique values



The above shows 56 unique patients, I will now join all into one csv document. And then try and find more data to include. The current issue is that there are multiple for some individuals. First I am not including activity as it provided information about the indiduduals movements and thats not an area I am focusing on

In [119]:
print("dfItem removed:")
dfItems.pop("activityDf")

dfItem removed:


,patient_id,location_name,date
0,0697d,Fridge Door,2019-06-28 13:03:29
1,0697d,Kitchen,2019-06-28 13:11:44
2,0697d,Front Door,2019-06-28 13:13:50
3,0697d,Bedroom,2019-06-28 13:13:53
4,0697d,Fridge Door,2019-06-28 13:14:09
...,...,...,...
1030554,fd100,Hallway,2019-06-30 23:48:50
1030555,fd100,Lounge,2019-06-30 23:49:40
1030556,fd100,Kitchen,2019-06-30 23:50:02
1030557,fd100,Front Door,2019-06-30 23:51:28


Now Activity is removed I can continue cleaning

In [120]:
#dropping dates as they arent needed
labelsDf = labelsDf.drop(columns=['date'])
physiologyDf = physiologyDf.drop(columns=['date'])
sleepDf = sleepDf.drop(columns=['date'])

In [121]:
labelsDf.head()

,patient_id,type
0,c55f8,Blood pressure
1,16f4b,Blood pressure
2,16f4b,Agitation
3,ec812,Blood pressure
4,16f4b,Agitation


In [122]:
#changing labelsdf to have multiple columns with 0 or 1 for each of the type conditions (this will make it easier to merge later)

labelsDf = (
    labelsDf
    .pivot_table(
        index='patient_id',
        columns='type',
        aggfunc='size',
        fill_value=0
    )
    .reset_index()
)
# Converting counts to presence (0/1) - this will be better for my models later on
label_cols = labelsDf.columns.drop('patient_id')
labelsDf[label_cols] = (labelsDf[label_cols] > 0).astype(int)

In [123]:
labelsDf.head()

type,patient_id,Agitation,Blood pressure,Body temperature,Body water,Pulse,Weight
0,0697d,0,1,0,0,1,0
1,099bc,1,1,0,0,0,0
2,0cda9,1,0,0,1,0,0
3,0d5ef,1,1,0,0,1,0
4,0efe8,0,0,0,1,1,0


Next I have expanded physiology by moving it to long format and grouping by patient_id - naming conventions and labelling will be done after merging

In [124]:
physiologyDf = (
    physiologyDf
    .pivot_table(
        index='patient_id',
        columns='device_type',
        values='value',
        aggfunc='mean'   # average per patient per device
    )
    .reset_index()
)

In [125]:
physiologyDf.head()

device_type,patient_id,Body Temperature,Body weight,Diastolic blood pressure,Heart rate,O/E - muscle mass,Skin Temperature,Systolic blood pressure,Total body water
0,0697d,36.386688,86.200000,80.000000,53.400000,64.550000,NaN,156.600000,50.900000
1,099bc,36.769330,52.614000,87.253333,69.226667,37.178378,NaN,151.466667,51.532432
2,0d5ef,36.572138,99.614286,88.266667,80.377778,67.933333,NaN,145.044444,48.633333
3,0efe8,36.370707,70.353731,75.800000,72.733333,39.607895,NaN,135.288889,42.381579
4,0f352,36.286647,72.400000,78.222222,74.000000,NaN,NaN,137.222222,NaN


In [126]:
sleepDf.head()

,patient_id,state,heart_rate,respiratory_rate,snoring
0,0f352,AWAKE,69.0,14.0,False
1,0f352,AWAKE,66.0,14.0,False
2,0f352,AWAKE,70.0,14.0,False
3,0f352,AWAKE,70.0,13.0,False
4,0f352,AWAKE,68.0,13.0,False


In [127]:
sleepDf = (
    sleepDf
    .pivot_table(
        index='patient_id',
        columns='state',
        aggfunc='size',
        fill_value=0
    )
    .reset_index()
)

# Converting counts to presence (0/1) - this will be better for my models later on (binary)
state_cols = sleepDf.columns.drop('patient_id')
sleepDf[state_cols] = (sleepDf[state_cols] > 0).astype(int)

In [128]:
sleepDf.head()

state,patient_id,AWAKE,DEEP,LIGHT,REM
0,0f352,1,1,1,1
1,16f4b,1,1,1,1
2,1fbe4,1,1,1,1
3,30a32,1,1,1,1
4,55cd4,1,1,1,1


In [129]:
#now all have been aggregated I can merge them

In [130]:
from functools import reduce #merging the new aggregated files with the demographics "patients" df

dfs = [demographicsDF, labelsDf, physiologyDf, sleepDf]

mergedDf = reduce(
    lambda left, right: pd.merge(left, right, on='patient_id', how='left'),
    dfs
)

In [131]:
mergedDf.head()

,patient_id,age,sex,Agitation,Blood pressure,Body temperature,Body water,Pulse,Weight,Body Temperature,...,Diastolic blood pressure,Heart rate,O/E - muscle mass,Skin Temperature,Systolic blood pressure,Total body water,AWAKE,DEEP,LIGHT,REM
0,b9d58,"(70, 80]",Female,1.0,0.0,0.0,0.0,0.0,0.0,36.330500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c55f8,"(80, 90]",Female,0.0,1.0,0.0,0.0,1.0,1.0,36.728481,...,82.730000,60.450000,35.636585,34.529511,144.98,47.571951,1.0,1.0,1.0,1.0
2,16f4b,"(80, 90]",Male,1.0,1.0,0.0,0.0,0.0,0.0,36.745277,...,88.785714,78.785714,NaN,34.586048,146.00,NaN,1.0,1.0,1.0,1.0
3,fd100,"(90, 110]",Female,0.0,1.0,0.0,0.0,0.0,0.0,36.627000,...,75.333333,56.333333,37.500000,NaN,148.00,48.700000,NaN,NaN,NaN,NaN
4,1fbe4,"(80, 90]",Male,0.0,1.0,0.0,0.0,1.0,0.0,36.034663,...,78.876923,54.215385,60.618919,33.088047,146.00,51.121622,1.0,1.0,1.0,1.0


In [132]:
outputPath = "/content/DementiaDetection/datasetFolder/mergedDataset.csv"
mergedDf.to_csv(outputPath, index=False)

# Chapter 2 - Looking into creating a synthetic dataset

## Why choose this?

As of limitations with gaining access to data I've decided to look into generating my own dataset with what I need based on field for questions and statistics produced by reputable sites like the NHS or .Gov

## Fields I will have:

1. **patientID** - random hash not repeatable

2. **Age** - 30-100 (decide later exact ages)

3. **Gender** - Male (0) or Female (1) (using binary where possible)

4. **dementiaHistory** - No(0) or Yes(1) - does the patient have a history of dementia in the family
5. **cognitionMedications** - No(0) or Yes(1) - is the patient on medications that could affect cognition
6. **cardiometabolic** - No(0) or Yes(1) - does the patient have conditions like hypertension, diabetes, stroke, or heart disease?
7. **Smoked** - No(0) or Yes(1) - has the patient ever smoked?
8. **Alchohol** - low(0) or moderate(1) or high(2) [ordinal datawise] - average alchohol consumption level
9. **physical** - low(0) or moderate(1) or high(2) [ordinal datawise] - average physical activity level
10. **sleep** - poor(0) or average(1) or good(2) [ordinal datawise] - average sleep stats for patient
11.** memory** - No (0) or Yes(1) - does the patient often forget things?
12. **mentalState** - No (0) or Yes(1) - does the patient experience any signs of depression or anxiety
13. **diet** - No (0) or Yes(1) - does the patient have a high carb diet
14. **bloodPressure** - No (0) or Yes(1) - does the patient have a high blood pressure
15. **Hearing** - No (0) or Yes(1) - does the patient have hearing loss
16. **Dementia** - percentage wise

The final dementia yes or no will be generated based on the statistics for the other columns using some form of linear model weighted on each. The probabilities are needed to be found and multiplied against each over for this generation.











## Researching the fields:

Percentage chances:

### Age

AlzheimersSociety (2022) and Office for Health Improvement & Disparities (2025)provided the AGE stats

**AGE:**
*WITH DEMENTIA*
1/14 overall population aged > 65
1/6 overall populationaged > 80
1/20 of people with dementia < 65 - important to understand this is with dementia

70800 people in the UK 2022 had early onset dementia out of 67,596,000 people in the UK general population in mid-2022. Population from the Office for National Statistics(2024).

$$
\frac{70800}{67,596,000} \times 100 = 0.104739925\% > 65 (2022)
$$
1/14 > 65 using AlzheimersSociety in 2022 = 7.14% of general population at that age

1/6 > 80 using AlzheimersSociety in 2022 = 16.67% general population at that age

Assuming > 65 < 80 = 7.14% as it was stated by AlzheimersSociety

AGE bracket | Percentage of having Dementia - these

| Age group | Percentage of having Dementia Approximately |
|-----------|-------------------|
| <65       | ~0.104% |
| >=65<80   | ~7.14%  |
| >=80      | ~16.67% |

FIND 60 PLUS 70 PLUS AND SO ON DO THIS!!!!!

# Chapter 3 - looking into more Datasets

### Whats the issue with synthetic data in training?
Synthetic data is not real and sometimes generated this means it can't be relied on for training. It seems like a good approach but data is always more informative if it is real. Therefore I have itterated and changed my approach:


Focus on alzheimer's as there is more data on this element of dementia
ADNI - most famous study
Focus on alzheimer's

Applied to
https://adni.loni.usc.edu/data-samples/adni-data/

https://adni.loni.usc.edu/help-faqs/adni-documentation/

### ADNI:

Part 1 could focus on Alzheimer's as it is easier to detect and there is more data on it (modelling)

Part 2 could focus on "if not Alzheimers what other dementia could it be?"

^this wont be based on a model this will just be a function based of some questions and gives a percentage back for each.

I have applied to ADNI as Senior Lecturer Dr Indranath Chatterjee advised it:

"The Alzheimer's Disease Neuroimaging Initiative (ADNI) is a longitudinal, multi-center, observational study.
The overall goal of ADNI is to validate biomarkers for Alzheimer's disease (AD) clinical trials." (Alzheimer's Disease Neuroimaging Initiative, 2026)



# Chapter 4 - ADNI

After being accepted to ADNI I explored and downloaded to datasets:

1. PTDEMOG (Participant Demographics): Very useful. Contains age, sex, education, ethnicity, and other baseline participant info
2. DXSUM (Diagnosis Summary): Crucial. This gives the clinical diagnosis of each participant at different visits: cognitively normal (CN), mild cognitive impairment (MCI), Alzheimer's disease (AD)

I will upload them to gitHub and then clean and merge them to then start my data analysis then modelling process





In [133]:
# Same proccess as before that I did for the smaller data set
DXSUMDF = pd.read_csv("/content/DementiaDetection/datasetFolder/ADNIDataset/DXSUM.csv")
PTDEMOGDF = pd.read_csv("/content/DementiaDetection/datasetFolder/ADNIDataset/PTDEMOG.csv")
MEDHISTDF = pd.read_csv("/content/DementiaDetection/datasetFolder/ADNIDataset/MEDHIST.csv")
PHYSICALDF = pd.read_csv("/content/DementiaDetection/datasetFolder/ADNIDataset/PHYSICAL.csv")
VITALSDF = pd.read_csv("/content/DementiaDetection/datasetFolder/ADNIDataset/VITALS.csv")

In [134]:
dfADNIItems = {"DXSUM":DXSUMDF,"PTDEMOG":PTDEMOGDF,"MEDHIST":MEDHISTDF,"PHYSICAL":PHYSICALDF,"VITALS":VITALSDF}
for name,df in dfADNIItems.items():#itterates through list and gives column names and how many unique values are in each
    print(f"{name}:")
    print("--------")
    print(df.columns.tolist())
    for col in df.columns:
        num_unique = df[col].nunique()
        print(f"{col}: {num_unique} unique values")
    print()

DXSUM:
--------
['PHASE', 'PTID', 'RID', 'VISCODE', 'VISCODE2', 'EXAMDATE', 'DIAGNOSIS', 'DXNORM', 'DXNODEP', 'DXMCI', 'DXMDES', 'DXMPTR1', 'DXMPTR2', 'DXMPTR3', 'DXMPTR4', 'DXMPTR5', 'DXMPTR6', 'DXMDUE', 'DXMOTHET', 'DXDSEV', 'DXDDUE', 'DXAD', 'DXAPP', 'DXAPROB', 'DXAPOSS', 'DXPARK', 'DXPDES', 'DXPCOG', 'DXPATYP', 'DXDEP', 'DXOTHDEM', 'DXODES', 'DXCONFID', 'ID', 'SITEID', 'USERDATE', 'USERDATE2', 'DD_CRF_VERSION_LABEL', 'LANGUAGE_CODE', 'HAS_QC_ERROR', 'update_stamp']
PHASE: 5 unique values
PTID: 3788 unique values
RID: 3788 unique values
VISCODE: 31 unique values
VISCODE2: 43 unique values
EXAMDATE: 4375 unique values
DIAGNOSIS: 3 unique values
DXNORM: 2 unique values
DXNODEP: 2 unique values
DXMCI: 2 unique values
DXMDES: 5 unique values
DXMPTR1: 3 unique values
DXMPTR2: 3 unique values
DXMPTR3: 4 unique values
DXMPTR4: 4 unique values
DXMPTR5: 3 unique values
DXMPTR6: 3 unique values
DXMDUE: 3 unique values
DXMOTHET: 18 unique values
DXDSEV: 4 unique values
DXDDUE: 2 unique values


As you can see the datasets have a lot of values, this will take time to clean, LETS GET STARTED!

In [135]:
#cleaning starting with DXSUMDF
DXSUMDF.head()

,PHASE,PTID,RID,VISCODE,VISCODE2,EXAMDATE,DIAGNOSIS,DXNORM,DXNODEP,DXMCI,...,DXODES,DXCONFID,ID,SITEID,USERDATE,USERDATE2,DD_CRF_VERSION_LABEL,LANGUAGE_CODE,HAS_QC_ERROR,update_stamp
0,ADNI1,011_S_0002,2,bl,bl,2005-09-29,1.0,1.0,-4.0,-4.0,...,-4.0,4.0,2,107,2005-10-01,NaN,NaN,NaN,NaN,2005-10-01 00:00:00
1,ADNI1,011_S_0003,3,bl,bl,2005-09-30,3.0,-4.0,-4.0,-4.0,...,-4.0,3.0,4,107,2005-10-01,NaN,NaN,NaN,NaN,2005-10-01 00:00:00
2,ADNI1,011_S_0005,5,bl,bl,2005-09-30,1.0,1.0,-4.0,-4.0,...,-4.0,4.0,6,107,2005-10-01,NaN,NaN,NaN,NaN,2005-10-01 00:00:00
3,ADNI1,011_S_0008,8,bl,bl,2005-09-30,1.0,1.0,-4.0,-4.0,...,-4.0,3.0,8,107,2005-10-01,NaN,NaN,NaN,NaN,2005-10-01 00:00:00
4,ADNI1,022_S_0007,7,bl,bl,2005-10-06,3.0,-4.0,-4.0,-4.0,...,-4.0,4.0,10,10,2005-10-06,NaN,NaN,NaN,NaN,2005-10-06 00:00:00


In [136]:
print(DXSUMDF['LANGUAGE_CODE'].unique()) #only nan (nothing) or e for english so can be dropped
print(DXSUMDF['LANGUAGE_CODE'].value_counts())

[nan 'e']
LANGUAGE_CODE
e    2431
Name: count, dtype: int64


In [137]:
print(DXSUMDF['HAS_QC_ERROR'].unique()) #no varability only one real value and so can be dropped
print(DXSUMDF['HAS_QC_ERROR'].value_counts())

[nan  0.]
HAS_QC_ERROR
0.0    2431
Name: count, dtype: int64


- update_stamp can also be dropped as it is just for recording when the database was updated - not useful for modelling
- DD_CRF_VERSION_LABEL has no varability and so is not useful for our model
- USERDATE2 is not clinical and tells us nothing about our patient so can be dropped
- same with USERDATE ^
- SITEID doesn't help determine dementia it just shows where ADNI scores for patient where recorded
- ID - is a unique identifier for rows not patients and could make our model learn the wrong idea - so we are dropping it

In [138]:
print(DXSUMDF['DXCONFID'].unique()) #stands for diagnosis confidence
print(DXSUMDF['DXCONFID'].value_counts())
#4 being very high, 3 being high, 2 being moderate and 1 being low confidence
#useful for our model so keeping it (can be removed later if not needed)

[ 4.  3.  1.  2. nan]
DXCONFID
4.0    2369
3.0    1244
2.0     222
1.0      33
Name: count, dtype: int64


In [139]:
#dropping columns so far...
DXSUMDFV2 = DXSUMDF.drop(["LANGUAGE_CODE","HAS_QC_ERROR","update_stamp","DD_CRF_VERSION_LABEL","USERDATE2","USERDATE",
 "SITEID","ID" ], axis=1)

In [140]:
DXSUMDFV2.head()

,PHASE,PTID,RID,VISCODE,VISCODE2,EXAMDATE,DIAGNOSIS,DXNORM,DXNODEP,DXMCI,...,DXAPROB,DXAPOSS,DXPARK,DXPDES,DXPCOG,DXPATYP,DXDEP,DXOTHDEM,DXODES,DXCONFID
0,ADNI1,011_S_0002,2,bl,bl,2005-09-29,1.0,1.0,-4.0,-4.0,...,-4,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,-4.0,4.0
1,ADNI1,011_S_0003,3,bl,bl,2005-09-30,3.0,-4.0,-4.0,-4.0,...,1,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,-4.0,3.0
2,ADNI1,011_S_0005,5,bl,bl,2005-09-30,1.0,1.0,-4.0,-4.0,...,-4,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,-4.0,4.0
3,ADNI1,011_S_0008,8,bl,bl,2005-09-30,1.0,1.0,-4.0,-4.0,...,-4,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,-4.0,3.0
4,ADNI1,022_S_0007,7,bl,bl,2005-10-06,3.0,-4.0,-4.0,-4.0,...,3,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,-4.0,4.0


- keeping Phase for now, but probably will drop later, just shows which ADNI phase each patient got recorded
- PTID - the patient ID - the unique identifier - this is the linker between
- RID = identifies the specific visit for that patient - so useful to have not dropping currently
- Viscodes and examdates - i want to explore more so not dropping yet

In [141]:
print(DXSUMDFV2['DIAGNOSIS'].unique()) #stands for diagnosis (VERY USEFUL NOT DROPPING)
print(DXSUMDFV2['DIAGNOSIS'].value_counts())

[ 1.  3.  2. nan]
DIAGNOSIS
2.0    6610
1.0    6369
3.0    3008
Name: count, dtype: int64



1.0 = 	CN (Cognitively Normal) | 2.0	= MCI (Mild Cognitive Impairment) | 3.0	= AD (Alzheimer's Disease) | nan	= Missing / not assessed

In [142]:
print(DXSUMDFV2['DXNORM'].unique())
print(DXSUMDFV2['DXNORM'].value_counts())
#might drop later as it could overcomplicate processing and has a lot of non entered
#details
# basically represents cognitively normal (1) vs non or not assessed

[ 1. -4. nan]
DXNORM
-4.0    2738
 1.0    1130
Name: count, dtype: int64


In [143]:
print(DXSUMDFV2['DXNODEP'].unique()) #this is a depression related column, it would be useful
#but its mainly missing so dropping it is probably a good choice (unless I want to add it later)
print(DXSUMDFV2['DXNODEP'].value_counts())

[-4.  1. nan]
DXNODEP
-4.0    3783
 1.0      85
Name: count, dtype: int64


In [144]:
print(DXSUMDFV2['DXMCI'].unique()) #Mild cognitive impairment
print(DXSUMDFV2['DXMCI'].value_counts())

[-4.  1. nan]
DXMCI
-4.0    2267
 1.0    1601
Name: count, dtype: int64


1 meaning the patient does and -4 meaning not assessed, can be derived from Diagnosis column so dropping it

In [145]:
print(DXSUMDFV2['DXMDES'].unique()) #Mild cognitive impairment subtype
print(DXSUMDFV2['DXMDES'].value_counts())

['-4' '1' '1|02' '2' '1|2' nan]
DXMDES
1       5983
-4      5482
1|2      296
1|02     167
2        124
Name: count, dtype: int64


In [146]:
print(DXSUMDFV2['DXODES'].unique()) #other disorders column very sparse dropping it
print(DXSUMDFV2['DXODES'].value_counts())

[-4. 14.  1. nan  4. 12.  6.  9.  7.]
DXODES
-4.0     3860
 14.0      29
 1.0       12
 12.0       6
 9.0        6
 4.0        3
 6.0        2
 7.0        1
Name: count, dtype: int64


subtypes overcomplicates the model so dropping it

In [148]:
DXSUMDFV3 = DXSUMDFV2.drop(["DXNODEP","DXMCI","DXMDES","DXODES"], axis=1)
DXSUMDFV3.head()

,PHASE,PTID,RID,VISCODE,VISCODE2,EXAMDATE,DIAGNOSIS,DXNORM,DXMPTR1,DXMPTR2,...,DXAPP,DXAPROB,DXAPOSS,DXPARK,DXPDES,DXPCOG,DXPATYP,DXDEP,DXOTHDEM,DXCONFID
0,ADNI1,011_S_0002,2,bl,bl,2005-09-29,1.0,1.0,-4.0,-4.0,...,-4.0,-4,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,4.0
1,ADNI1,011_S_0003,3,bl,bl,2005-09-30,3.0,-4.0,-4.0,-4.0,...,1.0,1,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,3.0
2,ADNI1,011_S_0005,5,bl,bl,2005-09-30,1.0,1.0,-4.0,-4.0,...,-4.0,-4,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,4.0
3,ADNI1,011_S_0008,8,bl,bl,2005-09-30,1.0,1.0,-4.0,-4.0,...,-4.0,-4,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,3.0
4,ADNI1,022_S_0007,7,bl,bl,2005-10-06,3.0,-4.0,-4.0,-4.0,...,1.0,3,-4,-4.0,-4.0,-4.0,-4.0,NaN,-4.0,4.0


# References

AlzheimersSociety (2022) What is dementia? Available at: https://www.alzheimers.org.uk/about-dementia/types-dementia/what-is-dementia(Accessed: 5 February 2026).

Office for Health Improvement & Disparities (2025) Dementia profile: prevalence and supporting well topics statistical commentary, March 2025. GOV.UK. Available at: https://www.gov.uk/government/statistics/dementia-profile-march-2025-update/dementia-profile-prevalence-and-supporting-well-topics-statistical-commentary-march-2025? (Accessed: 5 February 2026).

Office for National Statistics(2024) Population estimates for the UK, England, Wales, Scotland and Northern Ireland: mid-2022. Available at: https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/bulletins/annualmidyearpopulationestimates/mid2022 (Accessed: 5 February 2026).

Alzheimer's Disease Neuroimaging Initiative (ADNI) (2026) ADNI data. Available at: https://adni.loni.usc.edu/data-samples/adni-data/
(Accessed: 6 February 2026).